In [1]:
import pandas as pd
import os
import librosa
import numpy as np
import math
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import librosa.display

In [2]:
audio_dataset_path='C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/Al-Kauthar/'
metadata=pd.read_csv('C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/Al-Kauthar/metadata_108.csv')
metadata.head(20)

,slice_file_name,fold,classID,class
0,2_1.wav,4,108,Al-Kauthar
1,2_2.wav,4,108,Al-Kauthar
2,2_3.wav,4,108,Al-Kauthar
3,a.wav,2,108,Al-Kauthar
4,b.wav,2,108,Al-Kauthar
5,c.wav,2,108,Al-Kauthar
6,1_1.wav,3,108,Al-Kauthar
7,1_2.wav,3,108,Al-Kauthar
8,1_3.wav,3,108,Al-Kauthar
9,1.wav,1,108,Al-Kauthar


Feature Extraction

In [3]:
def mfcc_features_extractor_1sec(file):
    # audio, sample_rate = librosa.load(file_name) 
    # mfccs_features = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=40)
    # mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    y, sr = librosa.load(file, sr=16000)
    
    y = y[:192000]  #12 Secs for all audios
    zero_padding = np.zeros(192000 - y.shape[0], dtype=np.float32)
    y = np.concatenate([y, zero_padding])   

    # Define window size and hop length
    win_length = int(sr * 1) # 1 second window
    hop_length = int(win_length / 2) # 50% overlap

    # Extract MFCC features for each window
    mfccs = []
    for i in range(0, len(y)-win_length, hop_length):
        mfcc = librosa.feature.mfcc(y[i:i+win_length], sr=sr, n_mfcc=40)
        mfccs.append(mfcc)

    # Combine MFCC features for all windows
    mfccs = np.concatenate(mfccs, axis=1)
    mfccs = mfccs.T

    # Return final MFCC features
    return mfccs

    
    # return mfccs_scaled_features

In [4]:
# mfcc_extracted_features=[]

# for index_num,row in tqdm(metadata.iterrows()):
#     file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
#     final_class_labels=row["class"]
    
#     # MFCC Features Extraction
#     mfcc_data= mfcc_features_extractor_1sec(file_name)
#     mfcc_extracted_features.append([mfcc_data,final_class_labels])
#     # print(mfcc_extracted_features)

In [5]:
mfcc_data_by_fold = {}

# Iterate through the DataFrame and extract MFCC features
for index_num, row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path), 'fold' + str(row["fold"]) + '/', str(row["slice_file_name"]))
    final_class_labels = row["class"]

    # MFCC Features Extraction
    mfcc_data = mfcc_features_extractor_1sec(file_name)
    
    # Store the MFCC data in the dictionary by fold number
    fold_num = row["fold"]
    if fold_num not in mfcc_data_by_fold:
        mfcc_data_by_fold[fold_num] = {
            "data": [],
            "labels": []
        }
    mfcc_data_by_fold[fold_num]["data"].append(mfcc_data)
    mfcc_data_by_fold[fold_num]["labels"].append(final_class_labels)

# Now, convert the data in mfcc_data_by_fold to NumPy arrays
for fold_num, data_dict in mfcc_data_by_fold.items():
    data_dict["data"] = np.array(data_dict["data"])
    data_dict["labels"] = np.array(data_dict["labels"])

0it [00:00, ?it/s]C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.00486436  0.00673475  0.00485178 ... -0.09295268 -0.06584551
 -0.01902799] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.30245954  0.33469692  0.32966295 ... -0.2690308  -0.1085896
 -0.00173794] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[0.04010686 0.1026253  0.14761825 ... 0.02582019 0.04975414 0.07409305] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[0.05863062 0.1473877  0.2

In [6]:
fold_num = 1  # Replace with the fold number you're interested in
data_shape = mfcc_data_by_fold[fold_num]["data"].shape
labels_shape = mfcc_data_by_fold[fold_num]["labels"].shape

print(f"Shape of data for fold {fold_num}: {data_shape}")
print(f"Shape of labels for fold {fold_num}: {labels_shape}")

Shape of data for fold 1: (3, 704, 40)
Shape of labels for fold 1: (3,)


In [7]:
mfcc_data_by_fold[3]["data"].shape

(3, 704, 40)

In [8]:
# Iterate through the dictionary and reshape the data arrays for all fold_num values
for fold_num, data_dict in mfcc_data_by_fold.items():
    data_dict["data"] = data_dict["data"].reshape(data_dict["data"].shape + (1,))
    

In [9]:
mfcc_data_by_fold[1]["data"].shape

(3, 704, 40, 1)

After Training Work, Deep Features 

In [10]:
from keras.models import load_model
from keras.models import Model

# Load the saved model
loaded_model = load_model('Model_CNN2D_startwithnumpylengthsamemfcc_1sec_4032files_90_10_size_result 100_96.29.h5')

# Get the name of the intermediate layer from the loaded model summary
intermediate_layer_name = 'dense'

# Find the index of the intermediate layer in the model's layers list
intermediate_layer_index = [i for i, layer in enumerate(loaded_model.layers) if layer.name == intermediate_layer_name][0]

# Define a new model that includes layers up to the intermediate layer
intermediate_layer_model = Model(inputs=loaded_model.input,
                                  outputs=loaded_model.layers[intermediate_layer_index].output)

# List of inputs for each fold's MFCC data
input_data_list = [mfcc_data_by_fold[fold]["data"] for fold in range(1, 4)]  # Adjust the range as needed

# List to store intermediate layer outputs for each fold
intermediate_layer_outputs = []

# Iterate through the input data for each fold and predict the intermediate layer output
for input_data in input_data_list:
    intermediate_output = intermediate_layer_model.predict(input_data)
    intermediate_layer_outputs.append(intermediate_output)

# Now, intermediate_layer_outputs contains the intermediate layer outputs for each fold's input data


1/1 [==============================] - 0s 53ms/step


In [11]:
intermediate_layer_outputs = np.array(intermediate_layer_outputs)

In [12]:
intermediate_layer_outputs.shape

(3, 3, 1024)

In [13]:
intermediate_layer_outputs

array([[[-0.99998754, -1.        ,  0.9999993 , ...,  0.99878347,
         -0.99990934, -0.99917936],
        [-0.99596024, -1.        ,  0.9999796 , ...,  0.99684095,
         -0.9604286 , -0.98960346],
        [-0.9988143 , -1.        ,  0.9999668 , ...,  0.9975928 ,
         -0.99412966, -0.99999124]],

       [[-0.8736672 ,  0.08026168,  0.9956639 , ...,  0.99836016,
         -0.72621214, -0.8745978 ],
        [-0.9559751 ,  0.87666535,  0.9998687 , ...,  0.9851229 ,
         -0.9936816 , -0.9997451 ],
        [-0.9824648 ,  0.999865  ,  0.8631964 , ...,  0.9948172 ,
         -0.9859189 , -0.9934083 ]],

       [[-1.        ,  0.99823284,  0.99999833, ...,  0.99998325,
         -0.99997115, -1.        ],
        [-0.83630455, -0.49919653,  0.99997365, ...,  0.9983633 ,
         -0.9972119 , -0.994444  ],
        [-0.99971265,  0.6612473 ,  0.999997  , ...,  0.9988514 ,
         -0.9996611 , -0.999964  ]]], dtype=float32)

In [14]:
mean_across_folds = np.mean(intermediate_layer_outputs, axis=0)

In [15]:
al_kauthar_mean = mean_across_folds

In [16]:
al_kauthar_mean.shape

(3, 1024)

In [17]:
al_kauthar_mean

array([[-0.95788497,  0.02616485,  0.9985538 , ...,  0.9990423 ,
        -0.90869755, -0.95792574],
       [-0.9294133 , -0.2075104 ,  0.9999407 , ...,  0.99344236,
        -0.983774  , -0.9945975 ],
       [-0.99366397,  0.22037077,  0.9543867 , ...,  0.9970872 ,
        -0.99323654, -0.9977879 ]], dtype=float32)

In [18]:
# Specify the file path where you want to save the .npy file
file_path = 'al_kauthar_mean.npy'

# Save the array as a .npy file
np.save(file_path, al_kauthar_mean)

Testing of Single Audio

In [19]:
file_name = 'C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/kauthar_testing.wav'


mfcc_extracted_features_single=[]

mfcc_extracted_features_single= mfcc_features_extractor_1sec(file_name)

mfcc_extracted_features_single = np.array(mfcc_extracted_features_single)

mfcc_extracted_features_single = mfcc_extracted_features_single.reshape((1, 704, 40, 1))



C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.00152552  0.00116163  0.00017591 ... -0.12924773  0.00674683
 -0.01606241] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.15719876  0.09156487  0.19996221 ...  0.0250823  -0.00936632
 -0.08430377] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.12272218 -0.05589324  0.00633147 ...  0.07222988  0.0099933
 -0.18117386] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.08548963 -0.14062576 -0.1501341  

In [20]:
mfcc_extracted_features_single

array([[[[-4.72773041e+02],
         [ 7.71996002e+01],
         [-2.99716711e+00],
         ...,
         [-2.38157701e+00],
         [ 1.83951998e+00],
         [ 1.20611382e+01]],

        [[-4.54543457e+02],
         [ 7.57433777e+01],
         [-1.25619564e+01],
         ...,
         [-5.85508704e-01],
         [ 3.38528442e+00],
         [ 1.55222988e+01]],

        [[-4.61998291e+02],
         [ 6.94766235e+01],
         [-1.07766056e+01],
         ...,
         [ 9.29159403e-01],
         [ 4.79979610e+00],
         [ 1.58003235e+01]],

        ...,

        [[-1.13137085e+03],
         [ 0.00000000e+00],
         [ 0.00000000e+00],
         ...,
         [ 0.00000000e+00],
         [ 0.00000000e+00],
         [ 0.00000000e+00]],

        [[-1.13137085e+03],
         [ 0.00000000e+00],
         [ 0.00000000e+00],
         ...,
         [ 0.00000000e+00],
         [ 0.00000000e+00],
         [ 0.00000000e+00]],

        [[-1.13137085e+03],
         [ 0.00000000e+00],
         [

In [21]:
from keras.models import load_model
from keras.models import Model

# Load the saved model
loaded_model = load_model('Model_CNN2D_startwithnumpylengthsamemfcc_1sec_4032files_90_10_size_result 100_96.29.h5')

# Get the name of the intermediate layer from the loaded model summary
intermediate_layer_name = 'dense'

# Find the index of the intermediate layer in the model's layers list
intermediate_layer_index = [i for i, layer in enumerate(loaded_model.layers) if layer.name == intermediate_layer_name][0]

# Define a new model that includes layers up to the intermediate layer
intermediate_layer_model = Model(inputs=loaded_model.input,
                                  outputs=loaded_model.layers[intermediate_layer_index].output)

# Assuming you already have mfcc_extracted_features of shape (704, 40, 1)
# Replace this with your actual input data

# Predict the intermediate layer output for the input data
intermediate_output = intermediate_layer_model.predict(mfcc_extracted_features_single)

# Now, intermediate_output contains the intermediate layer output for mfcc_extracted_features


1/1 [==============================] - 0s 110ms/step


In [22]:
intermediate_output

array([[-0.9559753 ,  0.87666583,  0.9998686 , ...,  0.9851231 ,
        -0.9936816 , -0.999745  ]], dtype=float32)

In [23]:
intermediate_output.shape

(1, 1024)

In [24]:
intermediate_layer_outputs

array([[[-0.99998754, -1.        ,  0.9999993 , ...,  0.99878347,
         -0.99990934, -0.99917936],
        [-0.99596024, -1.        ,  0.9999796 , ...,  0.99684095,
         -0.9604286 , -0.98960346],
        [-0.9988143 , -1.        ,  0.9999668 , ...,  0.9975928 ,
         -0.99412966, -0.99999124]],

       [[-0.8736672 ,  0.08026168,  0.9956639 , ...,  0.99836016,
         -0.72621214, -0.8745978 ],
        [-0.9559751 ,  0.87666535,  0.9998687 , ...,  0.9851229 ,
         -0.9936816 , -0.9997451 ],
        [-0.9824648 ,  0.999865  ,  0.8631964 , ...,  0.9948172 ,
         -0.9859189 , -0.9934083 ]],

       [[-1.        ,  0.99823284,  0.99999833, ...,  0.99998325,
         -0.99997115, -1.        ],
        [-0.83630455, -0.49919653,  0.99997365, ...,  0.9983633 ,
         -0.9972119 , -0.994444  ],
        [-0.99971265,  0.6612473 ,  0.999997  , ...,  0.9988514 ,
         -0.9996611 , -0.999964  ]]], dtype=float32)

Euclidian Distance with Mean

In [25]:
import numpy as np

# Assuming you have al_fil_mean with shape (5, 1024) and intermediate_output with shape (1, 1024)
# Calculate the Euclidean distances between intermediate_output and each row in al_fil_mean
distances = np.linalg.norm(al_kauthar_mean - intermediate_output, axis=1)

# Find the index of the closest row in al_fil_mean
closest_index = np.argmin(distances)

# The value at closest_index in al_fil_mean is the closest to intermediate_output
closest_value = al_kauthar_mean[closest_index]

# Print the closest_index and closest_value
print(f"Index of closest value in al_kauthar_mean: {closest_index+1}")
print(f"Closest value in al_kauthar_mean: {closest_value}")

Index of closest value in al_fil_mean: 2
Closest value in al_fil_mean: [-0.9294133  -0.2075104   0.9999407  ...  0.99344236 -0.983774
 -0.9945975 ]


Euclidian Distance Without Mean

In [26]:
# # Reshape intermediate_output to match the shape of intermediate_layer_outputs
# intermediate_output_reshaped = intermediate_output.reshape((1, 1, 1024))

# # Initialize variables to keep track of the closest indices and distances
# closest_indices = None
# min_distances = None
# closest_set = None  # Initialize variable to track the closest set

# for i, intermediate_set in enumerate(intermediate_layer_outputs):
#     # Calculate the Euclidean distances between intermediate_output and intermediate_set
#     distances = np.linalg.norm(intermediate_set - intermediate_output_reshaped, axis=2)
    
#     # Find the index of the minimum distance
#     current_closest_index = np.argmin(distances)
    
#     # Update the closest index, distances, and closest set if this set is closer
#     if closest_indices is None or np.mean(distances) < np.mean(min_distances):
#         closest_indices = current_closest_index
#         min_distances = distances
#         closest_set = i

# # Print the closest index in intermediate_layer_outputs and which set it belongs to
# print(f"Closest index in intermediate_layer_outputs: {closest_indices}")
# if closest_set == 0:
#     print("It is closest to intermediate_layer_outputs[0]")
# elif closest_set == 1:
#     print("It is closest to intermediate_layer_outputs[1]")
# elif closest_set == 2:
#     print("It is closest to intermediate_layer_outputs[2]")